Download the data


In [ ]:
!git clone https://github.com/MaazAmjad/Pattern-Recognition-Course.git

Cloning into 'Pattern-Recognition-Course'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 1.36 MiB | 5.19 MiB/s, done.


Install necessary libraries


In [ ]:
# Import necessary libraries for data manipulation, visualization, and machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import math
import time

# Sklearn libraries for machine learning and text processing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# NLTK libraries for text processing (lemmatization, stemming, stopwords, POS tagging)
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Set up visualization
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Download necessary NLTK resources for text processing
nltk.download('wordnet')  # WordNet for lemmatization
nltk.download('omw-1.4')  # Open Multilingual Wordnet
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords for text cleaning
nltk.download('averaged_perceptron_tagger')  # POS tagger for part-of-speech tagging
nltk.download('tagsets_json')  # Tagset resource

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets_json.zip.


True

Assign train and test data to variables

In [ ]:
df_train = pd.read_csv("Pattern-Recognition-Course/data/train.csv")
df_test = pd.read_csv('Pattern-Recognition-Course/data/test.csv')
df_train.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
df_test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
#preprocessing process begins
df_train.describe()

,textID,text,selected_text,sentiment
count,27481,27480,27480,27481
unique,27481,27480,22463,3
top,6f7127d9d7,All this flirting going on - The ATG smiles...,good,neutral
freq,1,1,199,11118


In [ ]:
df_train.groupby('sentiment').describe()

textID                           text         \
           count unique         top freq  count unique   
sentiment                                                
negative    7781   7781  4f4c4fc327    1   7781   7781   
neutral    11118  11118  6f7127d9d7    1  11117  11117   
positive    8582   8582  ed167662a5    1   8582   8582   

                                                                   \
                                                         top freq   
sentiment                                                           
negative    I`ve wondered about rake to.  The client has ...    1   
neutral       All this flirting going on - The ATG smiles...    1   
positive                          But it was worth it  ****.    1   

          selected_text                       
                  count unique      top freq  
sentiment                                     
negative           7781   5861     miss  143  
neutral           11117  11111  morning    2  
positive           8582   5537     good  198

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
from nltk.corpus import stopwords

def text_process(tweet):
    """
    Cleans and processes a tweet string.

    This function takes a raw tweet string and performs several text cleaning steps:
    1. Checks if the input is a string; returns an empty string otherwise.
    2. Defines a list of common English stopwords and additional custom words to remove.
    3. Removes all punctuation from the tweet.
    4. Removes the defined stopwords from the tweet.
    5. Joins the remaining words back into a single cleaned string.

    Args:
        tweet (str): The input tweet string to be processed.

    Returns:
        str: The cleaned tweet string with punctuation and stopwords removed,
             or an empty string if the input was not a string.
    """
    # Handle non-string values gracefully
    if not isinstance(tweet, str):
        return ""

    # Define a set of stopwords, including standard English stopwords and some custom ones
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']

    # Remove punctuation from the tweet
    nopunc = ''.join([char for char in tweet if char not in string.punctuation])

    # Remove stopwords and join the words back into a string
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [ ]:
# Apply the function to the dataframe column
df_train['clean_tweets'] = df_train['text'].apply(text_process)
df_train.head()


,textID,text,selected_text,sentiment,clean_tweets
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,Id responded going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD miss San Diego
2,088c60f138,my boss is bullying me...,bullying me,negative,boss bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,Sons couldnt put releases already bought


In [ ]:
df_train['clean_tweets'] = df_train['text'].apply(text_process)
df_test['clean_tweets'] = df_test['text'].apply(text_process)

binary_mapping = {'negative':0, 'positive':1}

df_train = df_train[df_train['sentiment'].isin(binary_mapping.keys())].copy()

df_train['sentiment_num'] = df_train['sentiment'].map(binary_mapping)

In [ ]:
x = df_train['clean_tweets']
y = df_train['sentiment_num']

In [ ]:
x

,clean_tweets
1,Sooo SAD miss San Diego
2,boss bullying
3,interview leave alone
4,Sons couldnt put releases already bought
6,2am feedings baby fun smiles coos
...,...
27475,enjoy night
27476,wish could come see Denver husband lost job ca...
27477,Ive wondered rake client made clear NET force ...
27478,Yay good Enjoy break probably need hectic week...


In [ ]:
y

,sentiment_num
1,0
2,0
3,0
4,0
6,1
...,...
27475,1
27476,0
27477,0
27478,1


In [ ]:
x

,clean_tweets
1,Sooo SAD miss San Diego
2,boss bullying
3,interview leave alone
4,Sons couldnt put releases already bought
6,2am feedings baby fun smiles coos
...,...
27475,enjoy night
27476,wish could come see Denver husband lost job ca...
27477,Ive wondered rake client made clear NET force ...
27478,Yay good Enjoy break probably need hectic week...


In [ ]:
# prompt: split data into training and validation sets

x_train, x_val, y_train, y_val= train_test_split(x, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
x_train

,clean_tweets
26756,HUGE headache asprin work
6118,Respect Travis Clark Gone 75 Notches
16716,thx msg awesome luv new single xoxo httptrimkpDW
20824,Oh guess officially big nerd heard movie great...
8714,cant find jewellry dressssss
...,...
19307,wooooo tour started yay 13 days awesome time l...
25095,Thank much Rex FF
14378,wouldnt cost quite bit mean flying pizza China...
18957,perfect day throw back head kiss good bye love...


In [ ]:
x_val

,clean_tweets
19974,Nice place eat sao paulo brazil httptwitgoocom...
13691,gaah rain tomorrow
16915,Ive seen enough movies know something terrible...
4094,Hope roommates good morning without TP soap
26931,expensive
...,...
9649,awww love charlies enjoying lucky food LOL
1885,love Broughty Ferry
3522,3 days late happy birthday Hahahaha
26296,upset fact 2010 last year


In [ ]:
# prompt: instantiate CountVectorizer with binary= TRUE and fit on training data

vectorizer = CountVectorizer(binary=True)
x_train_dtm = vectorizer.fit_transform(x_train)
x_val_dtm = vectorizer.transform(x_val)


In [ ]:
clf = LogisticRegression()

In [ ]:
y_train

,sentiment_num
26756,0
6118,0
16716,1
20824,0
8714,0
...,...
19307,1
25095,1
14378,1
18957,1


In [ ]:
x_train_dtm

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 93716 stored elements and shape (13090, 17516)>

In [ ]:
clf = LogisticRegression(random_state=0).fit(x_train_dtm, y_train)

In [ ]:
clf.predict(x_val_dtm)

array([1, 0, 0, ..., 1, 0, 0])

In [ ]:
score = clf.score(x_val_dtm, y_val)
percentage_score = score * 100
print(f"Model score: {percentage_score:.2f}%")

Model score: 86.92%
